# Segmenting and Clustering Neighborhoods in Toronto

In [2]:
# using requests load given web-page into object page 
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

In [3]:
import pandas as pd

In [4]:
!pip install beautifulsoup4

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [34]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

# All requested packages already installed.



In [3]:
#here we parse page content
#page.content
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [1]:
#print(soup.prettify())

In [5]:
#Let's create dataframe for our data. It consists of 3 columns  
CACodeDf = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

In [6]:
#And finaly, let's load data from our page into dataframe  CACodeDf
i = 0
for tr in soup.find_all('tr')[1:]:
    tds = tr.find_all('td')  
    my_str = tds[2].text.replace('\n', '')
    CACodeDf.loc[i] = [tds[0].text]+ [tds[1].text] + [my_str]
    i = i +1
    print      (tds[0].text, tds[1].text, tds[2].text)

M1A Not assigned Not assigned

M2A Not assigned Not assigned

M3A North York Parkwoods

M4A North York Victoria Village

M5A Downtown Toronto Harbourfront

M5A Downtown Toronto Regent Park

M6A North York Lawrence Heights

M6A North York Lawrence Manor

M7A Queen's Park Not assigned

M8A Not assigned Not assigned

M9A Etobicoke Islington Avenue

M1B Scarborough Rouge

M1B Scarborough Malvern

M2B Not assigned Not assigned

M3B North York Don Mills North

M4B East York Woodbine Gardens

M4B East York Parkview Hill

M5B Downtown Toronto Ryerson

M5B Downtown Toronto Garden District

M6B North York Glencairn

M7B Not assigned Not assigned

M8B Not assigned Not assigned

M9B Etobicoke Cloverdale

M9B Etobicoke Islington

M9B Etobicoke Martin Grove

M9B Etobicoke Princess Gardens

M9B Etobicoke West Deane Park

M1C Scarborough Highland Creek

M1C Scarborough Rouge Hill

M1C Scarborough Port Union

M2C Not assigned Not assigned

M3C North York Flemingdon Park

M3C North York Don Mills South


IndexError: list index out of range

In [7]:
CACodeDf.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [8]:
CACodeDf.shape

(288, 3)

In [9]:
#Now , since we have dataframe filled with all necessary data, we should clear it from inappropriate rows
CACodeDf = CACodeDf[CACodeDf.Borough != "Not assigned"]
CACodeDf.head(10) 

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [10]:
CACodeDf.shape

(211, 3)

In [11]:
CACodeDf.loc[CACodeDf['Neighborhood'] == "Not assigned", ['Neighborhood']] = CACodeDf['Borough']
CACodeDf.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
CACodeDf.shape

(211, 3)

In [20]:
df = CACodeDf[['PostalCode','Borough']]
#df.drop_duplicates()
df=df.drop_duplicates(['PostalCode','Borough'])[['PostalCode','Borough']]
df['Neighborhood'] = df.apply(lambda row: "", axis=1)
df.head(10)
#df.shape

,PostalCode,Borough,Neighborhood
2,M3A,North York,
3,M4A,North York,
4,M5A,Downtown Toronto,
6,M6A,North York,
8,M7A,Queen's Park,
10,M9A,Etobicoke,
11,M1B,Scarborough,
14,M3B,North York,
15,M4B,East York,
17,M5B,Downtown Toronto,


In [30]:
for index, row in df.iterrows():
    my_str = ''
    for index2, row2 in CACodeDf.iterrows():
        if row2['PostalCode'] == row['PostalCode'] and row2['Borough'] == row['Borough']:
            my_str = row2['Neighborhood']+","+my_str
    row['Neighborhood'] = my_str[:-1]
df.head(30)     

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
6,M6A,North York,"Lawrence Manor,Lawrence Heights"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Malvern,Rouge"
14,M3B,North York,Don Mills North
15,M4B,East York,"Parkview Hill,Woodbine Gardens"
17,M5B,Downtown Toronto,"Garden District,Ryerson"
